In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained("UBC-NLP/MARBERT",local_files_only=False)
model = BertForMaskedLM.from_pretrained("UBC-NLP/MARBERT",local_files_only=False)


c:\Users\river\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from transformers import pipeline
fill = pipeline('fill-mask', model=model, tokenizer=tokenizer)

c:\Users\river\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
def getLoss(true_sentence ,masked_sentence):
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    # retrieve index of [MASK]
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    # print(logits.shape)
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    # print(predicted_token_id)
    labels = tokenizer(true_sentence, return_tensors="pt")["input_ids"]
    # mask labels of non-[MASK] tokens
    labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

    outputs = model(**inputs, labels=labels)
    return (tokenizer.decode(predicted_token_id), round(outputs.loss.item(), 2))

In [6]:
import numpy as np
def getTopPred(masked_sentence,topN = 100):
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, mask_token_index]    
    topN_tokend_id = np.argpartition(predicted_token_id.reshape(-1), -topN)[-topN:]
    return tokenizer.decode(topN_tokend_id)

In [10]:
getTopPred("دا مجرد رأى و[MASK] اعلم و ابقا")


'الناس❉ لن اقدرالله ربما الدين هنا دا بعدين ربي لكى هى فيما النتيجة والله البادي ربى نا انتا المولى فعلا سبحانه لم بكره الذى يارب الباقى هو الواقع بالله ده هم انتم فالله اكيد رب كان تعالى ربنا بس وحده ربكم الرب الرحمن رينا اللى وربنا ريي انا انت الشارع الحق اللہ ربناا لكن اكاد اللهم ما لله هوا انى نعم سوف الرسول الموفق لا الشرع احنا ان الله بكرة غدا ﷲ اللھ url اللهه هوه ربك القادم االله فقط نحن المستقبل واللة كنت الحاضر هما العباد رسوله الخالق حضرتك الل التاريخ عايز م الفعل طبعا انتى اللة'

In [13]:
getLoss(true_sentence="دا مجرد رأى و الله اعلم و ابقا ", masked_sentence="دا مجرد رأى و [MASK] اعلم و ابقا")

('الله', 0.11)

In [9]:
getLoss(true_sentence="دا مجرد رأى و الله اعلم و ابقا الله", masked_sentence="دا مجرد رأى و [MASK] اعلم و ابقا [MASK]")

('الله url', 2.9)

In [16]:
"دا مجرد رأى و الله اعلم و ابقا الله".replace("الله", " ".join(["[MASK]"]*1))

'دا مجرد رأى و [MASK] اعلم و ابقا [MASK]'

In [11]:
# دا مجرد رأى و الله اعلم و ابقا
fill("دا مجرد رأى و[MASK] اعلم و ابقا")

[{'score': 0.8920179605484009,
  'token': 1944,
  'token_str': 'ا ل ل ه',
  'sequence': 'دا مجرد راى و الله اعلم و ابقا'},
 {'score': 0.05682513117790222,
  'token': 2410,
  'token_str': 'ر ب ن ا',
  'sequence': 'دا مجرد راى و ربنا اعلم و ابقا'},
 {'score': 0.012048520147800446,
  'token': 2043,
  'token_str': 'ا ن ا',
  'sequence': 'دا مجرد راى و انا اعلم و ابقا'},
 {'score': 0.008924578316509724,
  'token': 2188,
  'token_str': 'و ا ل ل ه',
  'sequence': 'دا مجرد راى و والله اعلم و ابقا'},
 {'score': 0.0060320352204144,
  'token': 16954,
  'token_str': 'ر ب ى',
  'sequence': 'دا مجرد راى و ربى اعلم و ابقا'}]

In [12]:
getLoss(true_sentence="هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان مريضا أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا ", masked_sentence="هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان [MASK] أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا ")

('مسافر', 6.15)

## test strategy

In [17]:
"المشتروع" in tokenizer.vocab.keys()

False

In [18]:
# انا رايح اركب المشتروع
fill("انا رايح اركب [MASK]")

[{'score': 0.07754305005073547,
  'token': 12264,
  'token_str': 'ع ج ل',
  'sequence': 'انا رايح اركب عجل'},
 {'score': 0.05276244506239891,
  'token': 33856,
  'token_str': 'ت ا ك س ي',
  'sequence': 'انا رايح اركب تاكسي'},
 {'score': 0.04821603745222092,
  'token': 30165,
  'token_str': 'ا ل ق ط ر',
  'sequence': 'انا رايح اركب القطر'},
 {'score': 0.04384777322411537,
  'token': 60969,
  'token_str': 'ت و ك ت و ك',
  'sequence': 'انا رايح اركب توكتوك'},
 {'score': 0.04265390709042549,
  'token': 1936,
  'token_str': 'u r l',
  'sequence': 'انا رايح اركب url'}]

In [19]:
pred_top_100 = getTopPred("انا رايح اركب [MASK]")
pred_top_100

'اوتوستراد الهرم تاني قارب البوكيمون جنازة مواصلة فيراري موتو صاروخ واحد السيسي كفر دلوقتي القمر. السيارة الماتش المشروع اليكس التوكتوك مراجيح ترام حديد طيارات البحر ورا الصحراوي السياره بنزين الموج تقويم عربية عربيه بوكيمون الاسعاف سواق الزمالك مركب حافلة الجيش لوحدي مواصله ايه سفينة الشنطه الدايري [UNK] حشيش مكنة المحور تكسي بترول bmw الخيل الحصان حصان المرسيدس اهو الصاروخ المواصلات الهوا حمار العجله الميكروباص اتوبيس عجلة خيل ترحال مترو العربية طياره مرسيدس الموجه عجل مواصلات الطياره url القطر كريم الموجة العجلة العربيه طيارة قطار قطر الطيارة توكتوك اوبر موجة ميكروباص المترو الباص تاكسي باص جمل الاتوبيس القطار التاكسي عجله'

In [20]:
"المشروع" in tokenizer.vocab.keys()

True

In [21]:
"المشروع" in pred_top_100

True

In [22]:
point = 1

In [23]:
"المكتب" in pred_top_100

False

In [24]:
(predicted, loss_actual) = getLoss(true_sentence="انا رايح اركب المشروع", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_actual) 

('عجل', 6.78)

In [25]:
(predicted, loss_relevant) = getLoss(true_sentence="انا رايح اركب عجل", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_relevant) 

('عجل', 2.56)

In [26]:
(predicted, loss_not_relevant) = getLoss(true_sentence="انا رايح اركب المكتب", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_not_relevant) 

('عجل', 10.85)

In [27]:
# add award point to relevant distance because it appeared in the top 100 predicted words 
point = .5 * loss_actual 

loss_actual = loss_actual- point
loss_not_relevant = loss_not_relevant + point

actual_relevant_dist = abs( loss_actual - loss_relevant  )
actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

total_distance  = actual_relevant_dist + actual_not_relevant_dist

actual_relevant_precent = actual_not_relevant_dist  / total_distance
actual_notrelevant_precent = actual_relevant_dist / total_distance

print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")

Percentage of Relevance 92.89 %
Percentage of Not Relevant 7.11 %


In [28]:
actual_relevant_precent + actual_notrelevant_precent

1.0

## TEST 2

In [29]:
# After correcting "نرم" >> "نجم"
true_sentence = "بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم"
masked_sentence = "بقينا ف زمن اللي سب الدين مش ف لسانه ده [MASK]"

In [30]:
"نرم" in tokenizer.vocab.keys()

False

In [31]:
"نجم" in tokenizer.vocab.keys()

True

In [32]:
"نجم" in pred_top_100

False

In [33]:
pred_top_100 = getTopPred(masked_sentence)
pred_top_100

"😏😏 دلوقتي♨ - اخلاقنا 😞 عادي 💔 ارهاب 😓 👌🏻 ارحمونا غلط اخلاق تاني 😑😑 دينه خلاص بقى ✋ حقيقة 😂 الزمن قرف جهل ✋✋ واقع 😃 نفاق حالنا [UNK] ، حاله ☺ زمن كلام 😡 اسمه 😠. تخلف url6٨ 😀 مرض 😒😒 بقا كافر لعنه 😂😂 بجد 😅 زمان مجتمعنا خالص كله موضه النفاق كفر الطبيعي 👎 ؟ 👌☁ ستايل فرعون الواقع عيب هزار 😕 ابتلاء كمان حلال 😔 👊 حرام 😒'مصيبه🌟 حقيقه حوار 😑 😊 للاسف انتشر 😐 حقيقي اخره 😏! اسلوب والله مجتمع بقي بس هبل تويتر ✋🏻"

In [34]:
point = 0

In [35]:
"المكتب" in pred_top_100

False

In [36]:
fill(masked_sentence)

[{'score': 0.13026295602321625,
  'token': 16,
  'token_str': '.',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده.'},
 {'score': 0.11558369547128677,
  'token': 5,
  'token_str': '!',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده!'},
 {'score': 0.07561495900154114,
  'token': 1936,
  'token_str': 'u r l',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده url'},
 {'score': 0.04237668588757515,
  'token': 372,
  'token_str': '✋',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده ✋'},
 {'score': 0.020195290446281433,
  'token': 906,
  'token_str': '😒',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده 😒'}]

In [37]:
(predicted, loss_actual) = getLoss(true_sentence="بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم", masked_sentence= masked_sentence)
(predicted, loss_actual) 

('.', 11.2)

In [38]:
(predicted, loss_relevant) = getLoss(true_sentence=f"بقينا ف زمن اللي سب الدين مش ف لسانه ده {predicted}", masked_sentence=masked_sentence)
(predicted, loss_relevant) 

('.', 2.04)

In [39]:
(predicted, loss_not_relevant) = getLoss(true_sentence="بقينا ف زمن اللي سب الدين مش ف لسانه ده المكتب", masked_sentence=masked_sentence)
(predicted, loss_not_relevant) 

('.', 16.7)

In [40]:
loss_not_relevant

16.7

In [41]:
# add penality point to relevant distance because it didn't appeared in the top 100 predicted words 
point = .5 * loss_actual 

loss_actual = loss_actual + point
loss_not_relevant = loss_not_relevant - point

actual_relevant_dist = abs( loss_actual - loss_relevant  )
actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

total_distance  = actual_relevant_dist + actual_not_relevant_dist

actual_relevant_precent = actual_not_relevant_dist  / total_distance
actual_notrelevant_precent = actual_relevant_dist / total_distance

print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")

Percentage of Relevance 27.86 %
Percentage of Not Relevant 72.14 %


In [42]:
abs( loss_actual - loss_not_relevant  )

5.6999999999999975

In [43]:
abs( loss_actual - loss_relevant  )


14.759999999999998

In [45]:
type(tokenizer.vocab.keys()- pred_top_100.split())

set

In [46]:
tokenizer.tokenize("##ﻈﺮ")

['[UNK]', '[UNK]', 'ﻈ', '##ﺮ']

# Function

In [95]:
import random
import re 
def getRelevanceScore1(true_sentence, word):
    
    masked_sentence = true_sentence.replace(word,"[MASK]")
    pred_top_100 = getTopPred(masked_sentence)
    irrelevant_word = random.choices(list(tokenizer.vocab.keys()- pred_top_100.split()), k=1)[0]
    print("irrelevant_word in pred_top_100: ",irrelevant_word, irrelevant_word in pred_top_100)
    
    (predicted, loss_actual) = getLoss(true_sentence, masked_sentence)
    
    true_sentence2 = true_sentence
    # masked_sentence2 = masked_sentence

    # print(true_sentence2)
    for pred in predicted.split(" "):
        print(pred)
        true_sentence2 = true_sentence2.replace(word, pred,1)
        # print(true_sentence2)
        # masked_sentence2 = masked_sentence2.replace(word, "[MASK]",1)
    # (predicted, loss_relevant) = getLoss(true_sentence2, masked_sentence.replace(word, " ".join(["[MASK]"])))
    print( true_sentence2)
    # print(masked_sentence2) 
    (predicted, loss_relevant) = getLoss(true_sentence2, masked_sentence)

    n_tokens = len(tokenizer.tokenize(irrelevant_word))
    print("n_tokens: ",n_tokens)
    (predicted, loss_not_relevant) = getLoss(true_sentence.replace(word, irrelevant_word), masked_sentence = true_sentence.replace(word, " ".join(["[MASK]"]*n_tokens)))
    
    print("predicted", predicted)
    print("loss_actual", loss_actual)
    print("loss_relevant", loss_relevant)
    print("loss_not_relevant", loss_not_relevant)
    
    point = .5 * loss_actual 
    

    if word not in pred_top_100:
        point = -1* point
        print("Apply penality  = ", point)
    else:
        print("Apply reward  = ", point)
        
    loss_actual = loss_actual- point
    loss_not_relevant = loss_not_relevant + point
    
    actual_relevant_dist = abs( loss_actual - loss_relevant  )
    actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

    total_distance  = actual_relevant_dist + actual_not_relevant_dist

    actual_relevant_precent = actual_not_relevant_dist  / total_distance
    actual_notrelevant_precent = actual_relevant_dist / total_distance

    print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
    print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")
    return (actual_relevant_precent, actual_notrelevant_precent)


In [97]:

getRelevanceScore1("بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم","نجم")

irrelevant_word in pred_top_100:  وروح False
.
بقينا ف زمن اللي سب الدين مش ف لسانه ده .
n_tokens:  1
predicted .
loss_actual 11.2
loss_relevant 2.04
loss_not_relevant 17.55
Apply penality  =  -5.6
Percentage of Relevance 24.73 %
Percentage of Not Relevant 75.27 %


(0.24732279449260572, 0.7526772055073944)

In [98]:
getRelevanceScore1("انا رايح اركب المشروع","المشروع")

irrelevant_word in pred_top_100:  بنتيجة False
عجل
انا رايح اركب عجل
n_tokens:  1
predicted عجل
loss_actual 6.78
loss_relevant 2.56
loss_not_relevant 16.12
Apply reward  =  3.39
Percentage of Relevance 95.1 %
Percentage of Not Relevant 4.9 %


(0.951032448377581, 0.04896755162241888)

In [90]:
tokenizer.tokenize("الجامدة")

['الجامد', '##ة']

In [91]:
"الجامدة" in tokenizer.vocab.keys()

False